In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

import pandas as pd

output_correct = pd.DataFrame()
output_error = pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./part.xlsx',
                       header=0,
                       dtype=str).fillna('')

print('总数量：' + str(len(input_)))
print()

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
import UA
import Proxy

# = = = = = = = = = = = = = = =

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)
    
# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error
    
    while not work.empty():
        a = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        crawler_status = 'error'
        for b in range(100):
            try:
                resp = requests.get('https://uacparts.com/Catalog/Specs?part=' + input_.loc[a, 'UAC_Part_Number'],
                                    headers=UA.get_User_Agent_Requests(),
                                    proxies=Proxy.get_Proxy_Requests()).text
                
                # = = = = = = = = = = = = = = =

                soup = BeautifulSoup(resp.strip(), 'lxml')
                html = etree.HTML(str(soup))

                # = = = = = = = = = = = = = = =
                
                list_td = html.xpath('//table/tr/td[1]/text()')
                list_td_2 = html.xpath('//table/tr/td[2]/text()')

                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame([{'UAC_Part_Number': input_.loc[a, 'UAC_Part_Number']}])

                for td, td_2, in zip(list_td, list_td_2):
                    df_temp.loc[0, td.strip()] = td_2.strip()

                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

                # = = = = = = = = = = = = = = =
                
                crawler_status = 'ok'
                
                break
            except:
                time.sleep(0.3)
                continue
        
        # = = = = = = = = = = = = = = =
        
        if crawler_status == 'error':
            df_temp = pd.DataFrame([{'UAC_Part_Number': input_.loc[a, 'UAC_Part_Number']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')
            
        # = = = = = = = = = = = = = = =
        
        print('[' + crawler_status + '] - ' + input_.loc[a, 'UAC_Part_Number'] +  '\n[尝试次数：' + str(b+1) + '] - [剩余数量：' + str(work.qsize()) + '] - [当前时间：' + datetime.now().strftime('%H:%M:%S') + ']\n')
        
# = = = = = = = = = = = = = = =

list_task = []
for _ in range(20):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['UAC_Part_Number'], ascending=[True]).reset_index(drop=True)
output_correct.to_excel('./uac_specs-' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['UAC_Part_Number'], ascending=[True]).reset_index(drop=True)
    output_error.to_excel('./uac_specs_error.xlsx', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

总数量：1156

[ok] - CO 101210C
[尝试次数：1] - [剩余数量：1136] - [当前时间：19:09:01]

[ok] - CO 0073C
[尝试次数：1] - [剩余数量：1135] - [当前时间：19:09:01]

[ok] - CO 0040C
[尝试次数：1] - [剩余数量：1134] - [当前时间：19:09:01]

[ok] - CO 10076C
[尝试次数：1] - [剩余数量：1133] - [当前时间：19:09:01]

[ok] - CO 0011C
[尝试次数：1] - [剩余数量：1132] - [当前时间：19:09:01]

[ok] - CO 0041C
[尝试次数：1] - [剩余数量：1131] - [当前时间：19:09:01]

[ok] - CO 0071C
[尝试次数：1] - [剩余数量：1130] - [当前时间：19:09:01]

[ok] - CO 0074C
[尝试次数：1] - [剩余数量：1129] - [当前时间：19:09:01]

[ok] - CO 101200C
[尝试次数：1] - [剩余数量：1128] - [当前时间：19:09:01]

[ok] - CO 0024GLC
[尝试次数：1] - [剩余数量：1127] - [当前时间：19:09:01]

[ok] - CO 0042C
[尝试次数：1] - [剩余数量：1126] - [当前时间：19:09:01]

[ok] - CO 10082JC
[尝试次数：1] - [剩余数量：1125] - [当前时间：19:09:01]

[ok] - CO 10120C
[尝试次数：1] - [剩余数量：1124] - [当前时间：19:09:01]

[ok] - CO 0023GLC
[尝试次数：1] - [剩余数量：1123] - [当前时间：19:09:01]

[ok] - CO 10016C
[尝试次数：1] - [剩余数量：1122] - [当前时间：19:09:01]

[ok] - CO 0104JC
[尝试次数：1] - [剩余数量：1121] - [当前时间：19:09:01]

[ok] - CO 0076C
[尝试次数：1] - [剩余数量：1120] - [当前时间：1

[ok] - CO 10807JC
[尝试次数：5] - [剩余数量：998] - [当前时间：19:09:18]

[ok] - CO 10837C
[尝试次数：1] - [剩余数量：997] - [当前时间：19:09:19]

[ok] - CO 10840C
[尝试次数：2] - [剩余数量：996] - [当前时间：19:09:19]

[ok] - CO 10773T
[尝试次数：6] - [剩余数量：995] - [当前时间：19:09:19]

[ok] - CO 10801C
[尝试次数：3] - [剩余数量：994] - [当前时间：19:09:19]

[ok] - CO 10842AC
[尝试次数：2] - [剩余数量：993] - [当前时间：19:09:20]

[ok] - CO 10845AC
[尝试次数：2] - [剩余数量：992] - [当前时间：19:09:20]

[ok] - CO 10849T
[尝试次数：1] - [剩余数量：991] - [当前时间：19:09:20]

[ok] - CO 10854C
[尝试次数：1] - [剩余数量：990] - [当前时间：19:09:21]

[ok] - CO 10824JC
[尝试次数：4] - [剩余数量：989] - [当前时间：19:09:21]

[ok] - CO 10852AC
[尝试次数：2] - [剩余数量：988] - [当前时间：19:09:22]

[ok] - CO 10855C
[尝试次数：2] - [剩余数量：987] - [当前时间：19:09:22]

[ok] - CO 10856C
[尝试次数：2] - [剩余数量：986] - [当前时间：19:09:22]

[ok] - CO 10851AC
[尝试次数：1] - [剩余数量：985] - [当前时间：19:09:22]

[ok] - CO 10858AC
[尝试次数：3] - [剩余数量：984] - [当前时间：19:09:23]

[ok] - CO 10857C
[尝试次数：1] - [剩余数量：983] - [当前时间：19:09:23]

[ok] - CO 10865JC
[尝试次数：1] - [剩余数量：982] - [当前时间：19:09:24]

[ok] -

[ok] - 
[尝试次数：1] - [剩余数量：856] - [当前时间：19:10:14]

[ok] - CO 11162T
[尝试次数：1] - [剩余数量：855] - [当前时间：19:10:14]

[ok] - 
[尝试次数：1] - [剩余数量：854] - [当前时间：19:10:15]

[ok] - CO 11178JC
[尝试次数：1] - [剩余数量：853] - [当前时间：19:10:15]

[ok] - 
[尝试次数：1] - [剩余数量：852] - [当前时间：19:10:15]

[ok] - CO 10957C
[尝试次数：2] - [剩余数量：851] - [当前时间：19:10:15]

[ok] - CO 11182C
[尝试次数：1] - [剩余数量：850] - [当前时间：19:10:15]

[ok] - 
[尝试次数：1] - [剩余数量：849] - [当前时间：19:10:15]

[ok] - CO 11190C
[尝试次数：1] - [剩余数量：848] - [当前时间：19:10:16]

[ok] - 
[尝试次数：1] - [剩余数量：847] - [当前时间：19:10:16]

[ok] - CO 10985C
[尝试次数：5] - [剩余数量：846] - [当前时间：19:10:16]

[ok] - CO 11193C
[尝试次数：1] - [剩余数量：845] - [当前时间：19:10:16]

[ok] - CO 11194C
[尝试次数：1] - [剩余数量：844] - [当前时间：19:10:17]

[ok] - CO 11191C
[尝试次数：1] - [剩余数量：843] - [当前时间：19:10:17]

[ok] - CO 11192C
[尝试次数：1] - [剩余数量：842] - [当前时间：19:10:17]

[ok] - CO 11170MC
[尝试次数：1] - [剩余数量：841] - [当前时间：19:10:17]

[ok] - CO 11194X
[尝试次数：1] - [剩余数量：840] - [当前时间：19:10:17]

[ok] - CO 11200C
[尝试次数：1] - [剩余数量：839] - [当前时间：19:10:17]


[ok] - CO 11493C
[尝试次数：1] - [剩余数量：714] - [当前时间：19:10:40]

[ok] - CO 11490C
[尝试次数：3] - [剩余数量：713] - [当前时间：19:10:40]

[ok] - CO 11491C
[尝试次数：1] - [剩余数量：712] - [当前时间：19:10:40]

[ok] - CO 11498C
[尝试次数：1] - [剩余数量：711] - [当前时间：19:10:41]

[ok] - CO 11503C
[尝试次数：1] - [剩余数量：710] - [当前时间：19:10:42]

[ok] - CO 11499C
[尝试次数：3] - [剩余数量：709] - [当前时间：19:10:42]

[ok] - CO 11495C
[尝试次数：2] - [剩余数量：708] - [当前时间：19:10:42]

[ok] - CO 11509C
[尝试次数：2] - [剩余数量：707] - [当前时间：19:10:42]

[ok] - CO 11435C
[尝试次数：3] - [剩余数量：706] - [当前时间：19:10:42]

[ok] - CO 11501C
[尝试次数：3] - [剩余数量：705] - [当前时间：19:10:42]

[ok] - CO 11510C
[尝试次数：1] - [剩余数量：704] - [当前时间：19:10:43]

[ok] - CO 11516C
[尝试次数：1] - [剩余数量：703] - [当前时间：19:10:44]

[ok] - CO 11112C
[尝试次数：2] - [剩余数量：702] - [当前时间：19:10:44]

[ok] - CO 11514C
[尝试次数：2] - [剩余数量：701] - [当前时间：19:10:44]

[ok] - CO 11500C
[尝试次数：6] - [剩余数量：700] - [当前时间：19:10:44]

[ok] - CO 11529C
[尝试次数：2] - [剩余数量：699] - [当前时间：19:10:44]

[ok] - CO 11513C
[尝试次数：3] - [剩余数量：698] - [当前时间：19:10:45]

[ok] - CO 1153

[ok] - CO 21468LC
[尝试次数：1] - [剩余数量：572] - [当前时间：19:11:14]

[ok] - CO 21470JC
[尝试次数：1] - [剩余数量：571] - [当前时间：19:11:14]

[ok] - CO 21516JC
[尝试次数：1] - [剩余数量：570] - [当前时间：19:11:14]

[ok] - CO 21225C
[尝试次数：1] - [剩余数量：569] - [当前时间：19:11:15]

[ok] - CO 21471LC
[尝试次数：1] - [剩余数量：568] - [当前时间：19:11:15]

[ok] - CO 21519C
[尝试次数：1] - [剩余数量：567] - [当前时间：19:11:15]

[ok] - CO 21511LC
[尝试次数：1] - [剩余数量：566] - [当前时间：19:11:15]

[ok] - CO 21520C
[尝试次数：1] - [剩余数量：565] - [当前时间：19:11:15]

[ok] - CO 21532LC
[尝试次数：1] - [剩余数量：564] - [当前时间：19:11:15]

[ok] - CO 21571C
[尝试次数：1] - [剩余数量：563] - [当前时间：19:11:16]

[ok] - CO 21570C
[尝试次数：1] - [剩余数量：562] - [当前时间：19:11:17]

[ok] - CO 21578AC
[尝试次数：1] - [剩余数量：561] - [当前时间：19:11:17]

[ok] - CO 21579T
[尝试次数：1] - [剩余数量：560] - [当前时间：19:11:17]

[ok] - CO 21590C
[尝试次数：1] - [剩余数量：559] - [当前时间：19:11:17]

[ok] - CO 21577AC
[尝试次数：2] - [剩余数量：558] - [当前时间：19:11:17]

[ok] - CO 21584LC
[尝试次数：1] - [剩余数量：557] - [当前时间：19:11:17]

[ok] - CO 21224C
[尝试次数：1] - [剩余数量：556] - [当前时间：19:11:18]

[ok] 

[ok] - CO 29088C
[尝试次数：5] - [剩余数量：429] - [当前时间：19:11:46]

[ok] - CO 29087C
[尝试次数：5] - [剩余数量：428] - [当前时间：19:11:46]

[ok] - CO 29102C
[尝试次数：1] - [剩余数量：427] - [当前时间：19:11:46]

[ok] - CO 29100C
[尝试次数：3] - [剩余数量：426] - [当前时间：19:11:46]

[ok] - CO 29099C
[尝试次数：3] - [剩余数量：425] - [当前时间：19:11:47]

[ok] - CO 29105C
[尝试次数：1] - [剩余数量：424] - [当前时间：19:11:47]

[ok] - CO 20741C
[尝试次数：6] - [剩余数量：423] - [当前时间：19:11:48]

[ok] - CO 29113C
[尝试次数：1] - [剩余数量：422] - [当前时间：19:11:48]

[ok] - CO 29108C
[尝试次数：1] - [剩余数量：421] - [当前时间：19:11:48]

[ok] - CO 29107C
[尝试次数：1] - [剩余数量：420] - [当前时间：19:11:48]

[ok] - CO 29109C
[尝试次数：1] - [剩余数量：419] - [当前时间：19:11:48]

[ok] - CO 29111C
[尝试次数：1] - [剩余数量：418] - [当前时间：19:11:48]

[ok] - CO 29106C
[尝试次数：1] - [剩余数量：417] - [当前时间：19:11:48]

[ok] - CO 29114C
[尝试次数：1] - [剩余数量：416] - [当前时间：19:11:49]

[ok] - CO 29117C
[尝试次数：1] - [剩余数量：415] - [当前时间：19:11:49]

[ok] - CO 29116C
[尝试次数：1] - [剩余数量：414] - [当前时间：19:11:49]

[ok] - CO 29118C
[尝试次数：1] - [剩余数量：413] - [当前时间：19:11:49]

[ok] - CO 2912

[ok] - CO 30016C
[尝试次数：1] - [剩余数量：287] - [当前时间：19:12:23]

[ok] - CO 30019C
[尝试次数：1] - [剩余数量：286] - [当前时间：19:12:24]

[ok] - CO 30017C
[尝试次数：1] - [剩余数量：285] - [当前时间：19:12:24]

[ok] - CO 30022C
[尝试次数：1] - [剩余数量：284] - [当前时间：19:12:24]

[ok] - CO 3036AC
[尝试次数：1] - [剩余数量：283] - [当前时间：19:12:24]

[ok] - CO 30021C
[尝试次数：1] - [剩余数量：282] - [当前时间：19:12:24]

[ok] - CO 29104C
[尝试次数：2] - [剩余数量：281] - [当前时间：19:12:24]

[ok] - CO 29154C
[尝试次数：2] - [剩余数量：280] - [当前时间：19:12:25]

[ok] - CO 3057AC
[尝试次数：1] - [剩余数量：279] - [当前时间：19:12:25]

[ok] - CO 3407AC
[尝试次数：1] - [剩余数量：278] - [当前时间：19:12:25]

[ok] - CO 35109C
[尝试次数：1] - [剩余数量：277] - [当前时间：19:12:25]

[ok] - CO 35112C
[尝试次数：1] - [剩余数量：276] - [当前时间：19:12:26]

[ok] - CO 35108C
[尝试次数：1] - [剩余数量：275] - [当前时间：19:12:26]

[ok] - CO 29103C
[尝试次数：2] - [剩余数量：274] - [当前时间：19:12:26]

[ok] - CO 35114C
[尝试次数：1] - [剩余数量：273] - [当前时间：19:12:27]

[ok] - CO 3626AC
[尝试次数：1] - [剩余数量：272] - [当前时间：19:12:27]

[ok] - CO 4036C
[尝试次数：1] - [剩余数量：271] - [当前时间：19:12:27]

[ok] - CO 35110

[ok] - CO 4618C
[尝试次数：1] - [剩余数量：142] - [当前时间：19:13:09]

[ok] - CO 4261C
[尝试次数：2] - [剩余数量：141] - [当前时间：19:13:09]

[ok] - CO 4617C
[尝试次数：1] - [剩余数量：140] - [当前时间：19:13:09]

[ok] - CO 42070V
[尝试次数：2] - [剩余数量：139] - [当前时间：19:13:09]

[ok] - CO 4622C
[尝试次数：1] - [剩余数量：138] - [当前时间：19:13:09]

[ok] - CO 4626C
[尝试次数：1] - [剩余数量：137] - [当前时间：19:13:10]

[ok] - CO 4623C
[尝试次数：1] - [剩余数量：136] - [当前时间：19:13:10]

[ok] - CO 46234Q
[尝试次数：1] - [剩余数量：135] - [当前时间：19:13:10]

[ok] - CO 4630DKV
[尝试次数：1] - [剩余数量：134] - [当前时间：19:13:10]

[ok] - CO 4630DKQ
[尝试次数：1] - [剩余数量：133] - [当前时间：19:13:10]

[ok] - CO 4621DKV
[尝试次数：1] - [剩余数量：132] - [当前时间：19:13:11]

[ok] - CO 4615DKV
[尝试次数：1] - [剩余数量：131] - [当前时间：19:13:11]

[ok] - CO 4634DKV
[尝试次数：1] - [剩余数量：130] - [当前时间：19:13:11]

[ok] - CO 4643
[尝试次数：1] - [剩余数量：129] - [当前时间：19:13:11]

[ok] - CO 4634DKQ
[尝试次数：1] - [剩余数量：128] - [当前时间：19:13:11]

[ok] - CO 4637C
[尝试次数：1] - [剩余数量：127] - [当前时间：19:13:11]

[ok] - CO 4647
[尝试次数：1] - [剩余数量：126] - [当前时间：19:13:12]

[ok] - CO 4645C
[尝试

[ok] - CO YC 181C
[尝试次数：1] - [剩余数量：0] - [当前时间：19:13:39]

[ok] - CO 9777C
[尝试次数：1] - [剩余数量：0] - [当前时间：19:13:41]

[ok] - CO 4819C
[尝试次数：2] - [剩余数量：0] - [当前时间：19:13:59]

[ok] - CO 4803C
[尝试次数：2] - [剩余数量：0] - [当前时间：19:14:00]

[ok] - CO 6095C
[尝试次数：2] - [剩余数量：0] - [当前时间：19:14:02]

[ok] - CO 7702C
[尝试次数：2] - [剩余数量：0] - [当前时间：19:14:06]

[ok] - CO 48600V
[尝试次数：2] - [剩余数量：0] - [当前时间：19:14:07]

[ok] - CO 8147C
[尝试次数：2] - [剩余数量：0] - [当前时间：19:14:07]

[ok] - CO 6629
[尝试次数：4] - [剩余数量：0] - [当前时间：19:14:08]

[ok] - CO 4919AC
[尝试次数：2] - [剩余数量：0] - [当前时间：19:14:09]

[ok] - CO 7312C
[尝试次数：2] - [剩余数量：0] - [当前时间：19:14:11]

[ok] - CO 7819C
[尝试次数：5] - [剩余数量：0] - [当前时间：19:14:11]

[ok] - CO 7822C
[尝试次数：4] - [剩余数量：0] - [当前时间：19:14:12]

[ok] - CO 8044C
[尝试次数：4] - [剩余数量：0] - [当前时间：19:14:12]

[ok] - CO 6173C
[尝试次数：2] - [剩余数量：0] - [当前时间：19:14:22]

输出ing...

Done ~
